In [1]:
import mlblocks

In [4]:
primitives = [
    'sklearn.impute.SimpleImputer',
   'xgboost.XGBClassifier',
] 

In [5]:
pipeline = mlblocks.MLPipeline(primitives)

In [4]:
import numpy as np
print(np.__version__)

1.24.2


In [9]:
!pip show numpy

Name: numpy
Version: 1.18.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /opt/anaconda3/lib/python3.8/site-packages
Requires: 
Required-by: accelerate, awkward, blueqat, cirq-core, cvxpy, daal4py, dimod, dwave-neal, ecos, fastdtw, gensim, gwpy, h5py, imbalanced-learn, Keras, Keras-Preprocessing, lightgbm, lmfit, maggma, matminer, matplotlib, mendeleev, mkl-fft, mkl-random, ml-stars, mlblocks, mplhep, numba, opencv-python, opt-einsum, orion-ml, osqp, pandas, patsy, pymatgen, pyqsp, pyquil, pyts, qdldl, qiskit-aer, qiskit-aqua, qiskit-ibmq-provider, qiskit-ignis, qiskit-terra, Quandl, retworkx, robocrys, scikit-learn, scipy, scs, seaborn, spglib, statsmodels, tensorboard, tensorflow, transformers, uhi, uproot, xgboost, yfinance


In [6]:
from orion.data import load_signal

data = load_signal('exchange-2_cpm_results')
data.shape

(1624, 2)

In [2]:
#Load the signal and true anomalies
signal_name = 'M-7'
signal = load_signal(signal_name)
true_ano = load_anomalies(signal_name)

In [3]:
#Get values and indices of signal 
values = signal['value'].values
indices = signal.index.values

In [4]:
#make rolling windows
window_size = 2500
step_size = 500
windows, start_indices = rolling_window_sequences(values, indices, window_size, step_size)

In [5]:
#rolling window anomaly detection
final_ano = []
i = 0
for seq in windows: 
    #to print out where the error was thrown out
    i+= 1
    try:
        final_ano.append(get_anomalies(seq, create_message_zero_shot, get_gpt_model_response))
    except Exception as error:
        print(error)
        print(i)
        break

In [8]:
final_res = merge_anomaly_seq(final_ano, start_indices, window_size, step_size, beta = 0)

In [9]:
final_res

array([ 215,  224,  225,  227,  247,  248,  249,  250,  252,  253,  254,
        256,  562,  564,  566,  567,  568,  569,  570,  571,  572,  573,
        574,  575,  576,  577,  578,  579,  580,  581,  738,  739,  743,
       1444, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533,
       1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544,
       1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555,
       1691, 1692, 1693, 1694, 1695, 2068, 2069, 2070, 2071, 2072, 2073,
       2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084,
       2085, 2086, 2087, 2088, 2091, 2092, 2093, 2094, 2095, 2096, 2097,
       2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108,
       2109, 2110])

In [1]:
PROMPTS = {
    "system_message": "You are an exceptionally intelligent assistant that detect anomalies in time series data by listing all the anomalies.",
    "user_message": "Below is a [SEQUENCE], please return the anomalies in that sequence in [RESPONSE]. Only return the numbers. [SEQUENCE]"
}

In [2]:
text = '0, 11, 3'

In [6]:
message = ' '.join((PROMPTS['system_message'], PROMPTS['user_message'], text, '[RESPONSE]'))

In [7]:
message

'You are an exceptionally intelligent assistant that detect anomalies in time series data by listing all the anomalies. Below is a [SEQUENCE], please return the anomalies in that sequence in [RESPONSE]. Only return the numbers. [SEQUENCE] 0, 11, 3 [RESPONSE]'